In [1]:
#!/usr/bin/env python3
# -*- coding:utf-8 -*-
import os
import json
import re
import csv
from scipy.stats import chi2_contingency
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix,f1_score,roc_auc_score,roc_curve
import matplotlib.pyplot as plt
import math
np.random.seed(0)

/Users/linliu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def clean_str(s):  # remove puctuation
    s = re.sub('[^0-9\w\d\s]+', '', s).replace('\r'," ").replace('\n'," ").replace('\xa0',' ')
    return s.strip().lower()

def data_analysis(target_word_name=None):
    # Data Analysis

    word_list = {}

    #  ************** Target Word List ******************
    print("Get "+target_word_name+" Data Begining ......")
   
    target_words_num = {}
    target_words_list = []
    target_words_weight = {}

    if target_word_name == 'propaganda':

        fake_pro_data = []
        true_pro_data = []
        
        target_data_1 = open('../other_dict/task-1/task1.train.txt', encoding='utf-8', mode='r')
        target_data_1 = target_data_1.readlines()
        for line in target_data_1:
            line = line.strip('\n\r').lower().split('\t')
            text = line[0]
            label = line[2]
            if label == 'propaganda':
                true_pro_data.append(text)
            else:
                fake_pro_data.append(text)
                
        tesk_2_3_path = '../other_dict/tasks-2-3/train/'
        target_data_2_dirs = os.listdir(tesk_2_3_path)
        article_index_set = set()
        for file_name in target_data_2_dirs:
            an_article_index = file_name.split(".")[0] 
            article_index_set.add(an_article_index)
        #print("article_index_set:",article_index_set)

        for index in article_index_set:
            text_name = tesk_2_3_path + index + '.txt'
            try:
                text = open(text_name, encoding='utf-8', mode='r')
            except:
                print('No such file or directory: ', text_name)
                continue
            try:
                label_name = tesk_2_3_path + index + '.task2.labels'
                label = open(label_name, encoding='utf-8', mode='r')
            except:
                print('No such file or directory: ', label_name)
                continue
            text_list = []
            for line in text.readlines():
                text_list.append(line)
            label = label.readlines()
            label_list = []
            for line in label:
                label_list.append(line)
            for i in range(len(text_list)):
                #print(label_list[i])
                if i < len(label_list):
                    if text_list[i].strip() != '':
                        if "\tpropaganda" in label_list[i]:
                            true_pro_data.append(text_list[i])
                        elif "\tnon-propaganda" in label_list[i]:
                            fake_pro_data.append(text_list[i])
                        # else:
                            # print(label_list[i])
                            # print("No Match !!!", index, i)
        print(len(true_pro_data),len(fake_pro_data))

       
        fake_pro_num = 0 
        true_pro_num = 0
        word_list = {}
        for line in fake_pro_data:
            line_word_list = []
            fake_pro_num += 1
            line = line.strip('\n').split(' ')
            for word in line:
                word=clean_str(word)
                if word in line_word_list:
                    continue
                else:
                    line_word_list.append(word)
                    if word in word_list.keys():
                        word_list[word][0] += 1
                    else:
                        word_list[word] = [1, 0]

        for line in true_pro_data:
            line_word_list = []
            true_pro_num += 1
            line = line.strip('\n').split(' ')
            for word in line:
                if word in line_word_list:
                    continue
                else:
                    line_word_list.append(word)
                    if word in word_list.keys():
                        word_list[word][1] += 1
                    else:
                        word_list[word] = [0, 1]
        
        import nltk
        nltk.download('stopwords')
        from nltk.corpus import stopwords
        stop_words = stopwords.words('english')
        fake_word_dict={}
        true_word_dict={}
        for word, v in word_list.items():
            word_in_fake = v[0]
            word_in_true = v[1]
            if word_in_fake + word_in_true > 10 and word not in stop_words:
                # print(word_in_fake, word_in_true)
                word_not_in_fake = fake_pro_num - word_in_fake
                word_not_in_true = true_pro_num - word_in_true
                obs = np.array([[word_in_fake, word_not_in_fake], [word_in_true, word_not_in_true]])
                g, p, dof, expctd = chi2_contingency(obs)
                if(p < 0.05): 
                    if word_in_fake > word_in_true: 
                        fake_word_dict[word] = p
                    else:
                        true_word_dict[word] = p

    
    #  **************************************************

   
   
        

    fake_word_dict = sorted(fake_word_dict.items(), key=lambda d: d[1], reverse=False)  
    true_word_dict = sorted(true_word_dict.items(), key=lambda d: d[1], reverse=False)


   
    fake_target_words_path = './result_fake_true_week5/fake_' + target_word_name + '_words.csv'
    with open(fake_target_words_path, "w") as fake_key_words:
        writer = csv.writer(fake_key_words)
        for i, item in enumerate(fake_word_dict):
            clean_word = clean_str(item[0])
            try:
                word_weight = target_words_weight[clean_word]
            except:
                word_weight = 'None'
            writer.writerow([clean_word, item[1], word_weight])

    true_target_words_path = './result_fake_true_week5/true_' + target_word_name + '_words.csv'
    with open(true_target_words_path, "w") as true_key_words:
        writer = csv.writer(true_key_words)
        for i, item in enumerate(true_word_dict):
            clean_word = clean_str(item[0])
            try:
                word_weight = target_words_weight[clean_word]
            except:
                word_weight = 'None'
            writer.writerow([clean_word, item[1], word_weight])



data_analysis("propaganda")


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
words = stopwords.words('english')
print(words)
import os
path = "./result_fake_true_week5/"
files= os.listdir(path) 
s = []
for file in files: 
     if not os.path.isdir(file): 
        print("now...",file)
        if(file == '.DS_Store'):
            continue
        f = open(path+"/"+file); 
        iter_f = iter(f); 
        fw = open("./result_fake_true_reduce_stopwords_week5/"+file,'w+')
        reduce_list = []
        for line in iter_f: 
            word = line.split(',')[0]
            if word not in words:
                fw.write(line)
            else:
                reduce_list.append(word)
        print(reduce_list)
        f.close()
        fw.close()

Get propaganda Data Begining ......
7959 42297


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/linliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/linliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def load_propaganda_task2():
    '''
    output:[[sentence, label]]
    '''
    global_sentence_label = list()
    
    tesk_2_3_path = '../other_dict/tasks-2-3/train/'
    target_data_2_dirs = os.listdir(tesk_2_3_path)
    article_name_set = set()
    for file_name in target_data_2_dirs:
        article_name_set.add(file_name.split(".")[0])
    for index in list(article_name_set):
        text_name = tesk_2_3_path + index + '.txt'
        try:
            text = open(text_name, encoding='utf-8', mode='r')
        except:
            print('No such file or directory: ', text_name)
            continue
        label_name = tesk_2_3_path + index + '.task2.labels'
        try:
            label = open(label_name, encoding='utf-8', mode='r')
        except:
            print('No such file or directory: ', label_name)
            continue
        text = text.readlines()
        sentence_list = []
        for sentence in text:
            sentence_list.append(sentence)
        label = label.readlines()
        label_list = []
        for line in label:
            label_list.append(line)
        assert len(label_list)==len(sentence_list),"text name %s, len of text %d, \
                len of label %d"%(text_name, len(sentence_list), len(label_list))
        for index,sentence in enumerate(sentence_list):
            if "\tnon-propaganda" in label_list[index] and sentence.strip()!="":
                global_sentence_label.append([sentence.strip(),0])
            elif "\tpropaganda" in label_list[index] and sentence.strip()!="":
                global_sentence_label.append([sentence.strip(),1])
            else:
                continue
    return global_sentence_label
# load data
global_sentence_label = load_propaganda_task2()

In [4]:
pos_word_dict = dict()
neg_word_dict = dict()
pos_num = 0
neg_num = 0
for ele in global_sentence_label:
    if ele[1]==1:
        pos_num += 1
        word_list = set(ele[0].lower().split(" "))
        for word in word_list:
            if word in pos_word_dict:
                pos_word_dict[word] += 1
            else:
                pos_word_dict[word] = 1
    else:
        neg_num += 1
        word_list = set(ele[0].split(" "))
        for word in word_list:
            if word in neg_word_dict:
                neg_word_dict[word] += 1
            else:
                neg_word_dict[word] = 1
                

In [5]:
pos_word_set = set(pos_word_dict.keys())
neg_word_set = set(neg_word_dict.keys())
all_word_set = pos_word_set.union(neg_word_set)

In [6]:
pos_significant_word_set = set()
neg_significant_word_set = set()
for word in all_word_set:
    if word in pos_word_dict.keys():
        word_in_pos = pos_word_dict[word]
    else:
        word_in_pos = 0
    if word in neg_word_dict.keys():
        word_in_neg = neg_word_dict[word]
    else:
        word_in_neg = 0
        
    if word_in_pos + word_in_neg > 10:
        word_not_in_pos = pos_num - word_in_pos
        word_not_in_neg = neg_num - word_in_neg
        g, p, dof, expctd = chi2_contingency(np.array([[word_in_pos, word_not_in_pos],[word_in_neg, word_not_in_neg]]))
        if p < 0.05:
            if word_in_pos > word_in_neg:
                pos_significant_word_set.add(word)
            else:
                neg_significant_word_set.add(word)

In [7]:
print("num of pos significant words %d, num of neg significant word %d"%(len(pos_significant_word_set), len(neg_significant_word_set)))

num of pos significant words 491, num of neg significant word 792


In [8]:
target_word_name='nrc_emotion'
print("Get pos_"+target_word_name+" Data Begining ......")
nrc_word_dict = dict()
nrc_word_set=set()
if target_word_name == 'nrc_emotion':
        target_data = open('../other_dict/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', encoding='utf-8', mode='r')
        target_data = target_data.readlines()
        for line in target_data:
            line = line.strip('\n\r').lower().split('\t')
            if len(line)==3 and line[2]== "1" and line[0] in pos_significant_word_set:
              
                nrc_word_set.add(line[0])
        for word in nrc_word_set:
            nrc_word_dict[word] = 1            
print("len of pos nrm word dict is ", len(nrc_word_dict))

new_target_words_path = './result_fake_true_after_reduce_stopwords/true_' + target_word_name + '_words.csv'
with open(new_target_words_path, "w") as new_key_words:
    writer = csv.writer(new_key_words)
    for key in nrc_word_dict.keys():
        word = key
        try:
            word_weight = nrc_word_dict[word]
        except:
            word_weight = 'None'
        writer.writerow([word, word_weight])

target_word_name='nrc_emotion'
print("Get neg_"+target_word_name+" Data Begining ......")
nrc_word_dict = dict()
nrc_word_set=set()
if target_word_name == 'nrc_emotion':
        target_data = open('../other_dict/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', encoding='utf-8', mode='r')
        target_data = target_data.readlines()
        for line in target_data:
            line = line.strip('\n\r').lower().split('\t')
            if len(line)==3 and line[2]== "1" and line[0] in neg_significant_word_set:
                nrc_word_set.add(line[0])
        for word in nrc_word_set:
            nrc_word_dict[word] = 1            
print("len of neg nrm word dict is ", len(nrc_word_dict))

new_target_words_path = './result_fake_true_after_reduce_stopwords/fake_' + target_word_name + '_words.csv'
with open(new_target_words_path, "w") as new_key_words:
    writer = csv.writer(new_key_words)
    for key in nrc_word_dict.keys():
        word = key
        try:
            word_weight = nrc_word_dict[word]
        except:
            word_weight = 'None'
        writer.writerow([word, word_weight])

Get pos_nrc_emotion Data Begining ......
len of pos nrm word dict is  113
Get neg_nrc_emotion Data Begining ......
len of neg nrm word dict is  62


In [9]:
global_test_sentence_label = list()
sentence_test_list = []
label_test_list = []
with open('../other_dict/new_propaganda_test/task2test_rs.csv','r') as csvfile_test_test:
    reader_test = csv.reader(csvfile_test_test)
    for i,rows in enumerate(reader_test):
        row_test = rows
        text_test = row_test[3]
        label =row_test[2]
        if label=="non-propaganda":
            global_test_sentence_label.append([text_test,0])
        elif label=="propaganda":
            global_test_sentence_label.append([text_test,1])
        else:
            continue
# partition to positive/negative
global_test_sentence_label_positive = [pair for pair in global_test_sentence_label if pair[1]==1]
global_test_sentence_label_negative = [pair for pair in global_test_sentence_label if pair[1]==0]
print("num of sentence is %d"%(len(global_test_sentence_label)))
print("num of pos sentence is %d"%(len(global_test_sentence_label_positive)))
print("num of neg sentence is %d"%(len(global_test_sentence_label_negative)))
test_dataset = global_test_sentence_label_negative[:]
test_dataset.extend(global_test_sentence_label_positive[:])

num of sentence is 4032
num of pos sentence is 1239
num of neg sentence is 2793


In [10]:
def load_propaganda_task2():
    '''
    output:[[sentence, label]]
    '''
    global_sentence_label = list()
    
    tesk_2_3_path = '../other_dict/tasks-2-3/train/'
    target_data_2_dirs = os.listdir(tesk_2_3_path)
    article_name_set = set()
    for file_name in target_data_2_dirs:
        article_name_set.add(file_name.split(".")[0])
    for index in list(article_name_set):
        text_name = tesk_2_3_path + index + '.txt'
        try:
            text = open(text_name, encoding='utf-8', mode='r')
        except:
            print('No such file or directory: ', text_name)
            continue
        label_name = tesk_2_3_path + index + '.task2.labels'
        try:
            label = open(label_name, encoding='utf-8', mode='r')
        except:
            print('No such file or directory: ', label_name)
            continue
        text = text.readlines()
        sentence_list = []
        for sentence in text:
            sentence_list.append(sentence)
        label = label.readlines()
        label_list = []
        for line in label:
            label_list.append(line)
        assert len(label_list)==len(sentence_list),"text name %s, len of text %d, \
                len of label %d"%(text_name, len(sentence_list), len(label_list))
        for index,sentence in enumerate(sentence_list):
            if "\tnon-propaganda" in label_list[index] and sentence.strip()!="":
                global_sentence_label.append([sentence,0])
            elif "\tpropaganda" in label_list[index] and sentence.strip()!="":
                global_sentence_label.append([sentence,1])
            else:
                continue
    return global_sentence_label
# load data
global_sentence_label = load_propaganda_task2()
np.random.shuffle(global_sentence_label)
# partition to positive/negative
global_sentence_label_positive = [pair for pair in global_sentence_label if pair[1]==1]
global_sentence_label_negative = [pair for pair in global_sentence_label if pair[1]==0][:3938]
print("num of sentence is %d"%(len(global_sentence_label)))
print("num of pos sentence is %d"%(len(global_sentence_label_positive)))
print("num of neg sentence is %d"%(len(global_sentence_label_negative)))
train_dataset = global_sentence_label_negative[:]
train_dataset.extend(global_sentence_label_positive[:])

num of sentence is 14263
num of pos sentence is 3938
num of neg sentence is 3938


In [11]:
def load_dict(dict_path):
    word_set = set()
    dict_text = open(dict_path, encoding='utf-8', mode='r')
    for line in dict_text:
        if line.strip()!="":
            word =line.split(",")[0]
            word_set.add(word)
    return word_set
def word_num_in_dict(sentence, word_set):
    try:
        word_list = sentence.strip().split(" ")
    except:
        print(sentence)
    word_num = 0
    for word in word_list:
        if word in word_set:
            word_num += 1
    return word_num
def build_feature_matrix_and_label(dataset,dict_names,dict_path):
    num_sample = len(dataset)
    num_feature = len(dict_names)
    feature_matrix = np.zeros((num_sample, num_feature))
    label = np.zeros((num_sample,))
    for i in range(num_sample):
        for j in range(num_feature):
            if 'propaganda' in dict_names[j]:
                dict_path='./result_fake_true_week5/'
            else:
                dict_path="./result_fake_true_after_reduce_stopwords/"
            feature_matrix[i,j] = word_num_in_dict(dataset[i][0], load_dict(dict_path + dict_names[j] + "_words.csv"))
        label[i] = dataset[i][1]
    return feature_matrix,label
def build_feature_matrix_new_one_dict(dataset,dict_name,dict_path):
    if 'propaganda' in dict_name:
        dict_path='./result_fake_true_week5/'
    else:
        dict_path="./result_fake_true_after_reduce_stopwords/"
    word_list = list(load_dict(dict_path+dict_name+ "_words.csv"))
    num_sample = len(dataset)
    num_feature = len(word_list)
    feature_matrix = np.zeros((num_sample, num_feature))
    for i in range(len(dataset)):
        for j,word in enumerate(word_list):
            if word in dataset[i][0]:
                feature_matrix[i][j]=1
    return(feature_matrix)

dict_names = ['fake_persuasive','fake_sentiment','fake_nrc_emotion','fake_subjectivity','fake_technical','fake_all',
              'true_persuasive','true_sentiment','true_nrc_emotion','true_subjectivity','true_technical','true_all']
dict_path = "./result_fake_true_after_reduce_stopwords/"
train_feature_matrix, train_label = build_feature_matrix_and_label(train_dataset, dict_names, dict_path)
test_feature_matrix, test_label = build_feature_matrix_and_label(test_dataset, dict_names, dict_path)

In [12]:
# one-hot 
for dict_name in dict_names:
    train_feature_matrix=np.hstack((build_feature_matrix_new_one_dict(train_dataset,dict_name,dict_path),train_feature_matrix))
    test_feature_matrix=np.hstack((build_feature_matrix_new_one_dict(test_dataset,dict_name,dict_path),test_feature_matrix))

In [13]:
print("shape",train_feature_matrix.shape, train_label.shape, test_feature_matrix.shape, test_label.shape)

shape (7876, 1437) (7876,) (4032, 1437) (4032,)


In [14]:
#gbdt, train
gbdt = GradientBoostingClassifier(max_depth=4,
                                  random_state=0,
                                  min_samples_split=5,
                                  learning_rate=0.01,
                                  n_estimators=30,
                                  subsample=0.8)

sample_weight = np.zeros(train_label.shape,np.float)
sample_weight[train_label==1] = 1.0 / np.sum(train_label==1)
sample_weight[train_label==0] = 1.0 / np.sum(train_label==0)
rf = gbdt.fit(train_feature_matrix, train_label, sample_weight)

print("train================================")

val_score_rbf = gbdt.score(train_feature_matrix, train_label)#val_score_rbf = gbdt.score(x, y)
print("The train accuracy score of rf is : %f" % val_score_rbf)
predict_label = gbdt.predict(train_feature_matrix)
prob_y = gbdt.predict_proba(train_feature_matrix)[:,1]
cm = confusion_matrix(predict_label,train_label)
print("confusion matrix is ", cm)
#plot_confusion_matrix(cm, "GBDT Confusion Matrix")
print(" positive f1:",f1_score(train_label,predict_label,pos_label=True))
print(" negative f1:",f1_score(train_label,predict_label,pos_label=False))
#print('test_feature_matrix:',test_feature_matrix)
gbdt_auc = roc_auc_score(train_label, prob_y)
print('The AUC of GBDT: %.5f' % gbdt_auc)




print("test================================")
test_score_rbf = gbdt.score(test_feature_matrix, test_label)#test_score_rbf = gbdt.score(test_x, test_y)
print("The test accuracy score of rf is : %f" % test_score_rbf)
predict_label = gbdt.predict(test_feature_matrix)
prob_y = gbdt.predict_proba(test_feature_matrix)[:,1]
cm = confusion_matrix(predict_label,test_label)
print("confusion matrix is ", cm)
#plot_confusion_matrix(cm, "GBDT Confusion Matrix")


print(" positive f1:",f1_score(test_label,predict_label,pos_label=True))
print(" negative f1:",f1_score(test_label,predict_label,pos_label=False))

from sklearn.metrics import precision_score, recall_score, f1_score
f1 = f1_score(test_label, predict_label, average='binary')
print("f1:", f1 )
p = precision_score(test_label, predict_label, average='binary')
print("precision:", p)
r = recall_score(test_label, predict_label,average='binary')
print("recall:", r)


#print('test_feature_matrix:',test_feature_matrix)
gbdt_auc = roc_auc_score(test_label, prob_y)
print('The AUC of GBDT: %.5f' % gbdt_auc)

train================================
The train accuracy score of rf is : 0.620620
confusion matrix is  [[3102 2152]
 [ 836 1786]]
 positive f1: 0.5445121951219513
 negative f1: 0.674934725848564
The AUC of GBDT: 0.67317
test================================
The test accuracy score of rf is : 0.671379
confusion matrix is  [[2239  771]
 [ 554  468]]
 positive f1: 0.41397611676249446
 negative f1: 0.7716698259520938
f1: 0.41397611676249446
precision: 0.45792563600782776
recall: 0.37772397094430993
The AUC of GBDT: 0.62666
